In [1]:

!pip install ipython-autotime
%load_ext autotime

time: 3.6 ms (started: 2021-01-29 14:58:49 +00:00)


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn import metrics

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import pickle

import warnings
warnings.filterwarnings("ignore")

time: 1.24 s (started: 2021-01-29 14:58:49 +00:00)


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
time: 2.49 ms (started: 2021-01-29 14:58:50 +00:00)


In [4]:
#importing datasets
stores = pd.read_csv('/content/gdrive/MyDrive/CaseStudies/stores.csv')
features = pd.read_csv('/content/gdrive/MyDrive/CaseStudies/features.csv')
train = pd.read_csv('/content/gdrive/MyDrive/CaseStudies/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/CaseStudies/test.csv')

sample = pd.read_csv('/content/gdrive/MyDrive/CaseStudies/sampleSubmission.csv')

time: 493 ms (started: 2021-01-29 14:58:50 +00:00)


In [5]:
train_df = train.merge(stores, how='left').merge(features, how='left')
print(train_df.shape)
#print(train_df.head(2))
print("               ")
print(train_df.columns)

#Merging stores and features with Test data
test_df = test.merge(stores, how='left').merge(features, how='left')
print(test_df.shape)
#print(train_df.head(2))
print("               ")
print(test_df.columns)

train_df['Date'] = pd.to_datetime(train_df['Date'], format='%d-%m-%Y')
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%Y-%m-%d')

(421570, 16)
               
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Type', 'Size',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment'],
      dtype='object')
(115064, 15)
               
Index(['Store', 'Dept', 'Date', 'IsHoliday', 'Type', 'Size', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment'],
      dtype='object')
time: 397 ms (started: 2021-01-29 14:58:51 +00:00)


In [6]:
  '''
    This funtion takes pandas dataframe as input, perform preprocessing(replaceing the nan value with most counted class) etc.
    and then return pandas dataframe as output
    
    data : pandas dataframe
    return : pandas dataframe
    '''

'\n  This funtion takes pandas dataframe as input, perform preprocessing(replaceing the nan value with most counted class) etc.\n  and then return pandas dataframe as output\n  \n  data : pandas dataframe\n  return : pandas dataframe\n  '

time: 4.6 ms (started: 2021-01-29 14:58:51 +00:00)


In [7]:
def preprocessing(train_test):

 
  seasons_dict = {
    1:"Winter",
    2:"Winter",
    3:"Spring",
    4:"Spring",
    5:"Spring",
    6:"Summer",
    7:"Summer",
    8:"Summer",
    9:"Fall",
    10:"Fall",
    11:"Fall",
    12:"Winter"}

  train_test['Type'] = train_test['Type'].astype('category')
  train_test['IsHoliday'] = train_test['IsHoliday'].apply(lambda x: 0 if x == False else 1).astype('int64')
  train_test["Day"]= pd.DatetimeIndex(train_test['Date']).day
  train_test['Month'] = pd.DatetimeIndex(train_test['Date']).month
  train_test['Year'] = pd.DatetimeIndex(train_test['Date']).year
  train_test['MarkDown1'] = train_test['MarkDown1'].fillna(0)
  train_test['MarkDown2'] = train_test['MarkDown2'].fillna(0)
  train_test['MarkDown3'] = train_test['MarkDown3'].fillna(0)
  train_test['MarkDown4'] = train_test['MarkDown4'].fillna(0)
  train_test['MarkDown5'] = train_test['MarkDown5'].fillna(0)
  train_test['Season'] = train_test['Month'].map(seasons_dict).astype('category')
  train_test['Quarter'] = train_test['Date'].dt.quarter
  feature_drop=['Unemployment']
  train_test = train_test.drop(feature_drop, axis=1)
  train_test = pd.get_dummies(train_test, columns=['Type'])
  train_test['Week_Number'] = train_test['Date'].dt.isocalendar().week.astype('int')
  train_test = train_test.drop(['Date','Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3','MarkDown4', 'MarkDown5', 'CPI','Season'],axis=1)
 

  train_test = train_test.astype('int64')


  return train_test

time: 39.7 ms (started: 2021-01-29 14:58:51 +00:00)


In [8]:
y = train_df['Weekly_Sales']
train_df = train_df.drop(['Weekly_Sales'],axis=1)

train_df = preprocessing(train_df)
X_query = preprocessing(test_df)
train_df.loc[(train_df.Year==2010) & (train_df.Week_Number==13), 'IsHoliday'] = True
train_df.loc[(train_df.Year==2011) & (train_df.Week_Number==16), 'IsHoliday'] = True
train_df.loc[(train_df.Year==2012) & (train_df.Week_Number==14), 'IsHoliday'] = True
X_query.loc[(X_query.Year==2013) & (X_query.Week_Number==13), 'IsHoliday'] = True

X_train, X_valid, y_train, y_valid = train_test_split(train_df, y, test_size = 0.40)

time: 1.23 s (started: 2021-01-29 14:58:51 +00:00)


In [9]:
def predict(data):

  model = pickle.load(open('/content/gdrive/MyDrive/CaseStudies/model.pkl','rb'))
  pred = model.predict(data)
  return pred

time: 1.75 ms (started: 2021-01-29 14:58:53 +00:00)


In [10]:
def mae(y_true, y_pred):

  mae = mean_absolute_error(y_true, y_pred)

  return mae



time: 2.52 ms (started: 2021-01-29 14:58:53 +00:00)


In [13]:
pred_train = predict(X_train)

pred_test = predict(X_valid)

time: 41.1 s (started: 2021-01-29 15:27:07 +00:00)


In [14]:

print(metrics.mean_absolute_error(y_train,pred_train))

print(metrics.mean_absolute_error(y_valid,pred_test))

4167.325421925336
4118.114157357332
time: 14.2 ms (started: 2021-01-29 15:27:54 +00:00)
